In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

with open('/content/drive/MyDrive/LLM_Projects/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value



In [3]:
!pip install git+https://github.com/openai/whisper.git #install whisper getting from github
!sudo apt update && sudo apt install ffmpeg #ffmpeg allows us to work with audio and video files
!pip install pytube
!pip install   pydub -qq
!pip install pyannote.audio -qq
!pip install  transformers torch  numpy  -qq
!pip install groq gTTS







  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-preghnkx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-preghnkx
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-

In [4]:
!pip install elevenlabs -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.5 MB/s eta 0:00:00


**Task 1:Generating Q&A using GROQ from llama3 and outputting audio**

In [5]:
from pytube import YouTube

def download_youtube_audio(video_url):
    yt = YouTube(video_url)
    stream = yt.streams.filter(only_audio=True).first()
    audio_file_path = yt.title + ".mp3"

    stream.download(filename=audio_file_path)

    return audio_file_path


In [6]:
video_url = "https://youtu.be/Own9MiaP57o?si=6plLESUP5YD_1hBg"
audio_file_path = download_youtube_audio(video_url)

In [7]:

!whisper "/content/Apple Reveals iOS 18 at WWDC 2024.mp3" --model medium.en --output_format txt

original_output_file = "/content/Apple Reveals iOS 18 at WWDC 2024.txt"
new_output_file = "/content/Apple_WWDC_2024_Transcription.txt"

with open(original_output_file, 'r') as file:
    transcribed_text = file.read()

with open(new_output_file, 'w') as file:
    file.write(transcribed_text)

print(f"Transcription has been saved to {new_output_file}")


100%|█████████████████████████████████████| 1.42G/1.42G [05:18<00:00, 4.79MiB/s]
[00:00.000 --> 00:06.640]  iOS 18 is a big release that delivers more ways to customize your iPhone, stay connected,
[00:06.640 --> 00:11.200]  and relive special moments. You can already customize your home screen with your favorite
[00:11.200 --> 00:17.920]  wallpaper, apps, and widgets, letting your personality shine through. And now, your app
[00:17.920 --> 00:24.960]  icons and widgets can add even more. I have this photo I love as my wallpaper, and now I can
[00:24.960 --> 00:30.400]  continue to enjoy it when I unlock my iPhone, because I can arrange my apps and widgets to
[00:30.400 --> 00:37.200]  frame it perfectly. I can select them all and easily place them along the bottom, right above
[00:37.200 --> 00:45.840]  the dock, for easy access, or even off to the side. We have an awesome new look for app icons
[00:45.840 --> 00:54.800]  when we go into dark mode. Let's turn it on. Isn't that cool? N

In [8]:
import os
from groq import Groq

def get_completion(prompt, model="llama3-70b-8192"):
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

transcribed_file_path = "/content/Apple Reveals iOS 18 at WWDC 2024.txt"

with open(transcribed_file_path, 'r') as file:
    transcribed_text = file.read()

prompt = f"Create 5 Q&A pairs from the following text. Ensure that each pair includes only the question and the answer. Do not include any introductory phrases, headings, or concluding statements. Each question should be directly related to the content, and each answer should be concise and informative:\n\n{transcribed_text}"

q_and_a_text = get_completion(prompt)

lines = q_and_a_text.split('\n')
cleaned_q_and_a_text = []
for line in lines:
    line = line.strip()
    if line.startswith("Q: "):
        cleaned_q_and_a_text.append(line[3:].strip())
    elif line.startswith("A: "):
        cleaned_q_and_a_text.append(line[3:].strip())

if len(cleaned_q_and_a_text) % 2 != 0:
    raise ValueError("The number of lines in the Q&A text is not even, indicating a possible mismatch between questions and answers.")

q_and_a_output_file_path = "/content/Apple_WWDC_2024_Q_and_A.txt"
with open(q_and_a_output_file_path, 'w') as file:
    file.write("\n".join(cleaned_q_and_a_text))

print(f"Q&A pairs have been saved to {q_and_a_output_file_path}")

Q&A pairs have been saved to /content/Apple_WWDC_2024_Q_and_A.txt


In [9]:
import os
from io import BytesIO
from pydub import AudioSegment
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

def text_to_speech_stream(text: str, voice_id: str) -> BytesIO:
    response = client.text_to_speech.convert(
        voice_id=voice_id,
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_multilingual_v2",
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
        ),
    )
    audio_stream = BytesIO()
    for chunk in response:
        if chunk:
            audio_stream.write(chunk)
    audio_stream.seek(0)
    return audio_stream


with open(q_and_a_output_file_path, 'r') as file:
    q_and_a_text = file.read()

if not q_and_a_text.strip():
    raise ValueError("No text found in the Q&A file.")

lines = [line.strip() for line in q_and_a_text.split('\n') if line.strip()]
if len(lines) % 2 != 0:
    raise ValueError("The number of lines in the Q&A text is not even, indicating a possible mismatch between questions and answers.")

questions = [lines[i] for i in range(0, len(lines), 2)]
answers = [lines[i] for i in range(1, len(lines), 2)]

question_voice_id = "21m00Tcm4TlvDq8ikWAM"
answer_voice_id = "29vD33N1CtxCmqQRPOHJ"

def generate_audio_stream(text, voice_id):
    audio_stream = text_to_speech_stream(text, voice_id)
    return audio_stream

question_audio_streams = [generate_audio_stream(f" {question}", question_voice_id) for question in questions]
answer_audio_streams = [generate_audio_stream(f" {answer}", answer_voice_id) for answer in answers]

question_audio = AudioSegment.empty()
answer_audio = AudioSegment.empty()

combined_audio = AudioSegment.empty()

for question_stream, answer_stream in zip(question_audio_streams, answer_audio_streams):
    question_segment = AudioSegment.from_file(question_stream)
    answer_segment = AudioSegment.from_file(answer_stream)
    combined_audio += question_segment
    combined_audio += answer_segment

audio_output_file_path = "/content/Apple_WWDC_2024_Q_and_A_combined.mp3"
combined_audio.export(audio_output_file_path, format="mp3")

print(f"Combined Q&A audio saved to: {audio_output_file_path}")


Combined Q&A audio saved to: /content/Apple_WWDC_2024_Q_and_A_combined.mp3


**Task 2: Identifying the number of speakers and gender of the speakers for a given youtube video**

In [10]:
from pytube import YouTube
from pydub import AudioSegment
from pyannote.audio import Pipeline
import os


video_url = "https://www.youtube.com/watch?v=yX5EJf4R77s"
yt = YouTube(video_url)
stream = yt.streams.filter(only_audio=True).first()
audio_file_path = stream.download(filename="panel_discussion_audio.mp4")

audio = AudioSegment.from_file(audio_file_path, format="mp4")
audio_path = "panel_discussion_audio.wav"
audio.export(audio_path, format="wav")

print(f"Audio has been downloaded and converted to WAV format: {audio_path}")

Audio has been downloaded and converted to WAV format: panel_discussion_audio.wav


In [15]:

import os
import torch
import torchaudio
import numpy as np
from pydub import AudioSegment
from pyannote.audio import Pipeline
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Wav2Vec2Processor

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_KEY")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=HUGGING_FACE_TOKEN
)

audio_file = "panel_discussion_audio.wav"

diarization = pipeline(audio_file)

audio = AudioSegment.from_wav(audio_file)

output_dir = "k"
os.makedirs(output_dir, exist_ok=True)

speaker_segments = {}

for turn, _, speaker in diarization.itertracks(yield_label=True):
    start_time = turn.start * 1000  # pydub works in milliseconds
    end_time = turn.end * 1000
    speaker_audio = audio[start_time:end_time]

    if speaker not in speaker_segments:
        speaker_segments[speaker] = [speaker_audio]
    else:
        speaker_segments[speaker].append(speaker_audio)

audio_paths = []
for speaker, segments in speaker_segments.items():
    combined_audio = segments[0]
    for segment in segments[1:]:
        combined_audio += segment
    output_file = os.path.join(output_dir, f"speaker_{speaker}.wav")
    combined_audio.export(output_file, format="wav")
    audio_paths.append(output_file)
    print(f"Saved {output_file}")


Saved k/speaker_SPEAKER_03.wav
Saved k/speaker_SPEAKER_01.wav
Saved k/speaker_SPEAKER_00.wav
Saved k/speaker_SPEAKER_02.wav


In [17]:
import torchaudio
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification
import tqdm
from torch.utils.data import DataLoader


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, sampling_rate=16000, max_audio_len=5):
        self.dataset = dataset
        self.sampling_rate = sampling_rate
        self.max_audio_len = max_audio_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        filepath = self.dataset[index]
        speech_array, sr = torchaudio.load(filepath)
        if speech_array.shape[0] > 1:
            speech_array = torch.mean(speech_array, dim=0, keepdim=True)
        if sr != self.sampling_rate:
            transform = torchaudio.transforms.Resample(sr, self.sampling_rate)
            speech_array = transform(speech_array)
        len_audio = speech_array.shape[1]
        if len_audio < self.max_audio_len * self.sampling_rate:
            padding = torch.zeros(1, self.max_audio_len * self.sampling_rate - len_audio)
            speech_array = torch.cat([speech_array, padding], dim=1)
        else:
            speech_array = speech_array[:, :self.max_audio_len * self.sampling_rate]
        speech_array = speech_array.squeeze().numpy()
        return {"input_values": speech_array, "attention_mask": None}

class CollateFunc:
    def __init__(self, processor, padding=True, sampling_rate=16000):
        self.processor = processor
        self.padding = padding
        self.sampling_rate = sampling_rate

    def __call__(self, batch):
        input_values = [item["input_values"] for item in batch]
        batch = self.processor(input_values, sampling_rate=self.sampling_rate, return_tensors="pt", padding=self.padding)
        return {"input_values": batch.input_values, "attention_mask": batch.attention_mask}

def predict(test_dataloader, model, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm.tqdm(test_dataloader):
            input_values = batch['input_values'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_values, attention_mask=attention_mask).logits
            scores = torch.nn.functional.softmax(logits, dim=-1)
            pred = torch.argmax(scores, dim=1).cpu().detach().numpy()
            preds.extend(pred)
    return preds

def get_gender(model_name_or_path, audio_paths, label2id, id2label, device):
    num_labels = 2
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
    model = AutoModelForAudioClassification.from_pretrained(
        model_name_or_path, num_labels=num_labels, label2id=label2id, id2label=id2label
    )
    test_dataset = CustomDataset(audio_paths, max_audio_len=5)
    data_collator = CollateFunc(processor=feature_extractor, padding=True, sampling_rate=16000)
    test_dataloader = DataLoader(test_dataset, batch_size=16, collate_fn=data_collator, shuffle=False, num_workers=2)
    preds = predict(test_dataloader=test_dataloader, model=model, device=device)
    return preds

model_name_or_path = "alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
label2id = {"female": 0, "male": 1}
id2label = {0: "female", 1: "male"}

audio_paths = []
segments_dir = "segments"
os.makedirs(segments_dir, exist_ok=True)

speakers = {}
speaker_id = 0

for turn, _, speaker in diarization.itertracks(yield_label=True):
    start_time = turn.start * 1000
    end_time = turn.end * 1000
    segment_audio = audio[start_time:end_time]

    segment_path = os.path.join(segments_dir, f"segment_{start_time}_{end_time}.wav")
    segment_audio.export(segment_path, format="wav")
    audio_paths.append(segment_path)

    if speaker not in speakers:
        speakers[speaker] = {
            "id": speaker_id,
            "segments": []
        }
        speaker_id += 1

    speakers[speaker]["segments"].append(segment_path)

preds = get_gender(model_name_or_path, audio_paths, label2id, id2label, device)

for speaker in speakers:
    speaker_gender_counts = {"male": 0, "female": 0}
    for segment in speakers[speaker]["segments"]:
        segment_index = audio_paths.index(segment)
        predicted_gender = id2label[preds[segment_index]]
        speaker_gender_counts[predicted_gender] += 1
    speakers[speaker]["gender"] = max(speaker_gender_counts, key=speaker_gender_counts.get)

male_count = 0
female_count = 0

for speaker in speakers:
    gender = speakers[speaker]["gender"]
    if gender == "male":
        male_count += 1
    elif gender == "female":
        female_count += 1

print(f"Number of male speakers: {male_count}")
print(f"Number of female speakers: {female_count}")
print(f"Total number of speakers: {male_count + female_count}")
print("Gender identification completed for all speakers.")


Some weights of the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at alefiury/wav2vec2-large-xlsr-53-gender-recognition-librispeech and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.p

Number of male speakers: 3
Number of female speakers: 1
Total number of speakers: 4
Gender identification completed for all speakers.
